In [4]:
import sys
sys.path.append('../ry')
from libry import *
from numpy import *

In [5]:
K = Config()
K.addFile("../rai-robotModels/pr2/pr2.g")
K.addFile("../models/tables.g")

K.addFrame("obj0", "table1", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(0 0 .15)>" )
K.addFrame("obj1", "table1", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(0 .2 .15)>" )
K.addFrame("obj2", "table1", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(0 .4 .15)>" )
K.addFrame("obj3", "table1", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(0 .6.15)>" )
K.addFrame("tray", "table2", "type:ssBox size:[.15 .15 .04 .02] color:[0. 1. 0.], logical={ table }, Q:<t(0 0 .07)>" );
K.addFrame("", "tray", "type:ssBox size:[.27 .27 .04 .02] color:[0. 1. 0.]" )
V = K.view()

In [6]:
lgp = K.lgp("../models/fol-pickAndPlace.g")
lgp.nodeInfo()
# this writes the initial state, which is important to check:
# do the grippers have the gripper predicate, do all objects have the object predicate, and tables the table predicate? These need to be set using a 'logical' attribute in the g-file
# the on predicate should automatically be generated based on the configuration

{'boundsConstraints': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsCost': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsFeasible': [1, 1, 1, 1, 1, 1],
 'decision': '<ROOT>',
 'path': '',
 'state': 'STATE_0(START_STATE):{(gripper pr2R):True, (gripper pr2L):True, (table table2):True, (object obj0):True, (object obj1):True, (object obj2):True, (object obj3):True, (table tray):True, (partOf table2 tray):True, (on obj0 table1):True, (on obj1 table1):True, (on obj2 table1):True, (on obj3 table1):True}'}

In [7]:
lgp.getDecisions()
# This is also useful to check: inspect all decisions possible in this node, which expands the node.
# If there is no good decisions, the FOL rules are buggy

['(grasp pr2R obj0)',
 '(grasp pr2R obj1)',
 '(grasp pr2R obj2)',
 '(grasp pr2R obj3)',
 '(grasp pr2L obj0)',
 '(grasp pr2L obj1)',
 '(grasp pr2L obj2)',
 '(grasp pr2L obj3)']

In [8]:
lgp.walkToDecision(3)
lgp.nodeInfo()
# Using getDecisions and walkToDecision and walkToParent, you can walk to anywhere in the tree by hand

{'boundsConstraints': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsCost': [0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsFeasible': [1, 1, 1, 1, 1, 1],
 'decision': '(grasp pr2R obj3)',
 'path': '(grasp pr2R obj3) ',
 'state': 'STATE_4(STATE_0):{(gripper pr2R):True, (gripper pr2L):True, (table table2):True, (object obj0):True, (object obj1):True, (object obj2):True, (object obj3):True, (table tray):True, (partOf table2 tray):True, (on obj0 table1):True, (on obj1 table1):True, (on obj2 table1):True, (on obj3 table1):True, decision(grasp pr2R obj3):True, (grasped pr2R obj3):True, (held obj3):True, (busy pr2R):True, komo(touch pr2R obj3):True, (stable pr2R obj3):True}'}

In [9]:
lgp.viewTree()

In [10]:
lgp.walkToNode("(grasp pr2R obj0) (grasp pr2L obj1) (place pr2R obj0 tray)")
lgp.nodeInfo()

{'boundsConstraints': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsCost': [0.30000000000000004, 0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsFeasible': [1, 1, 1, 1, 1, 1],
 'decision': '(place pr2R obj0 tray)',
 'path': '(grasp pr2R obj0) (grasp pr2L obj1) (place pr2R obj0 tray) ',
 'state': 'STATE_15(STATE_9):{(gripper pr2R):True, (gripper pr2L):True, (table table2):True, (object obj0):True, (object obj1):True, (object obj2):True, (object obj3):True, (table tray):True, (partOf table2 tray):True, (on obj0 table1):True, (on obj1 table1):True, (on obj2 table1):True, (on obj3 table1):True, (grasped pr2L obj1):True, (held obj1):True, (busy pr2L):True, (stable pr2L obj1):True, decision(place pr2R obj0 tray):True, (placed obj0 tray):True, (on obj0 tray):True, komo(touch pr2R obj0):True, (notheld obj0):True, block(INFEASIBLE grasp ANY obj0):True, komo(above obj0 tray):True, (stableOn tray obj0):True}'}

In [11]:
# at a node, you can compute bounds, namely BT.seq (just key frames), BT.path (the full path),
# and BT.setPath (also the full path, but seeded with the BT.seq result)
lgp.optBound(BT.seq, True)
lgp.nodeInfo()

{'boundsConstraints': [0.0, 0.0, 0.0032115672861163964, 0.0, 0.0, 0.0],
 'boundsCost': [0.30000000000000004, 0.0, 0.3148222643599409, 0.0, 0.0, 0.0],
 'boundsFeasible': [1, 1, 1, 1, 1, 1],
 'decision': '(place pr2R obj0 tray)',
 'path': '(grasp pr2R obj0) (grasp pr2L obj1) (place pr2R obj0 tray) ',
 'state': 'STATE_15(STATE_9):{(gripper pr2R):True, (gripper pr2L):True, (table table2):True, (object obj0):True, (object obj1):True, (object obj2):True, (object obj3):True, (table tray):True, (partOf table2 tray):True, (on obj0 table1):True, (on obj1 table1):True, (on obj2 table1):True, (on obj3 table1):True, (grasped pr2L obj1):True, (held obj1):True, (busy pr2L):True, (stable pr2L obj1):True, decision(place pr2R obj0 tray):True, (placed obj0 tray):True, (on obj0 tray):True, komo(touch pr2R obj0):True, (notheld obj0):True, block(INFEASIBLE grasp ANY obj0):True, komo(above obj0 tray):True, (stableOn tray obj0):True}'}

In [12]:
komo = lgp.getKOMOforBound(BT.seq)
komo.display()

In [13]:
komo=0

In [14]:
lgp.optBound(BT.path, True)
lgp.nodeInfo()

{'boundsConstraints': [0.0,
  0.0,
  0.0032115672861163964,
  0.031238032217880213,
  0.0,
  0.0],
 'boundsCost': [0.30000000000000004,
  0.0,
  0.3148222643599409,
  2.3963050901119347,
  0.0,
  0.0],
 'boundsFeasible': [1, 1, 1, 1, 1, 1],
 'decision': '(place pr2R obj0 tray)',
 'path': '(grasp pr2R obj0) (grasp pr2L obj1) (place pr2R obj0 tray) ',
 'state': 'STATE_15(STATE_9):{(gripper pr2R):True, (gripper pr2L):True, (table table2):True, (object obj0):True, (object obj1):True, (object obj2):True, (object obj3):True, (table tray):True, (partOf table2 tray):True, (on obj0 table1):True, (on obj1 table1):True, (on obj2 table1):True, (on obj3 table1):True, (grasped pr2L obj1):True, (held obj1):True, (busy pr2L):True, (stable pr2L obj1):True, decision(place pr2R obj0 tray):True, (placed obj0 tray):True, (on obj0 tray):True, komo(touch pr2R obj0):True, (notheld obj0):True, block(INFEASIBLE grasp ANY obj0):True, komo(above obj0 tray):True, (stableOn tray obj0):True}'}

In [12]:
lgp.viewTree()

In [13]:
# finally, the full multi-bound tree search (MBTS)
# you typically want to add termination rules, i.e., symbolic goals
print("THIS RUNS A THREAD. CHECK THE CONSOLE FOR OUTPUT. THIS IS GENERATING LOTS OF FILES.")
lgp.addTerminalRule("(on obj0 tray) (on obj1 tray) (on obj2 tray)")
lgp.run(2)

THIS RUNS A THREAD. CHECK THE CONSOLE FOR OUTPUT. THIS IS GENERATING LOTS OF FILES.


In [16]:
# wait until you have some number of solutions found (repeat executing this line...)
lgp.numSolutions()

6

In [17]:
# query the optimization features of the 0. solution
lgp.getReport(0, BT.seqPath)

[{'description': 'qItself:ALL',
  'scale': 0.01,
  'sos_value': 311.7586037813418,
  'type': 'sos'},
 {'feature': 'Transition',
  'scale': 1.0,
  'sos_value': 5.7627225710742875,
  'type': 'sos'},
 {'description': 'QuaternionNorms',
  'scale': 3.0,
  'sos_value': 2.6632289146511483e-08,
  'type': 'sos'},
 {'description': 'qItself:ALL',
  'scale': 0.1,
  'sos_value': 17.153935748334586,
  'target': [-0.5740060146016434,
   1.2973250801277876,
   0.6109984890503489,
   0.10022732628204395,
   0.017727921664832856,
   0.0074533251120756875,
   0.06326422184254125,
   0.5394272241388459,
   0.40016654821805525,
   0.37290068370807233,
   0.3526814231414411,
   -0.8895910522857582,
   1.0092934683688293,
   -0.9347986071108706,
   -1.9951059105087443,
   -2.0955185934254237,
   1.495328108352624,
   -0.26331314961800056,
   -0.4762406336462278,
   -0.39143469060400515,
   0.4980697392311131,
   -0.021210868144617877,
   0.08292596142376496,
   -0.07838777465295627,
   -0.48588729345126,
   

In [20]:
# get the KOMO object for the seqPath computation of the 0. solution
komo = lgp.getKOMO(0, BT.seqPath)

In [21]:
komo.displayTrajectory() #SOOO SLOOOW (TODO: add parameter for display speed)

In [22]:
# assign K to the 20. configuration of the 0. solution, check display
# you can now query anything (joint state, frame state, features)
X = komo.getConfiguration(20)
K.setFrameState(X) 

In [19]:
lgp.stop() #stops the thread... takes a while to finish the current job

In [19]:
lgp.run(2) #will continue where it stopped

In [ ]:
komo=0
lgp=0

In [1]:
import sys
sys.path.append('../ry')
from libry import *

K = Config()
D = K.view()

K.addFile('../test/lgp-example.g');

lgp = K.lgp("../test/fol.g");

In [2]:
lgp.walkToNode("(grasp baxterR stick) (push stickTip redBall table1) (grasp baxterL redBall) ");
print(lgp.nodeInfo())

{'boundsFeasible': [1, 1, 1, 1, 1, 1], 'state': 'STATE_50(STATE_46):{(table table1):True, (object redBall):True, (gripper baxterR):True, (object baxterR):True, (moves baxterR):True, (gripper baxterL):True, (object baxterL):True, (moves baxterL):True, (object stick):True, (object stickTip):True, (pusher stickTip):True, (partOf stick stickTip):True, (partOf stickTip stick):True, (on redBall table1):True, (on stick table1):True, (on stickTip table1):True, (grasped baxterR stick):True, (held stick):True, (animate stick):True, (busy baxterR):True, (stable baxterR stick):True, (held stickTip):True, (animate stickTip):True, (animate redBall):True, decision(grasp baxterL redBall):True, (grasped baxterL redBall):True, (held redBall):True, (busy baxterL):True, komo(flagClear redBall):True, komo(touch baxterL redBall):True, (stable baxterL redBall):True, komo(liftDownUp baxterL):True, (QUIT):True}', 'decision': '(grasp baxterL redBall)', 'boundsCost': [0.30000000000000004, 0.0, 0.0, 0.0, 0.0, 0.0

In [3]:
lgp.optBound(BT.pose, True);

RuntimeError: objective.cpp:setCostSpecs:17(-2) CHECK_GE failed: 'toStep'=1 'fromStep'=2 -- 

In [ ]:
komo = lgp.getKOMOforBound(BT.path)
komo.display()

input("Press Enter to continue...")